<a href="https://colab.research.google.com/github/M-Amrollahi/Personal-Notes/blob/master/ML-notes/selfsupervised_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torchvision import datasets
from IPython.display import display
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader,Dataset
import torch.nn as nn
import sklearn.metrics as metr
from sklearn.decomposition import PCA
from collections import OrderedDict

In [2]:
dsmnist = datasets.MNIST("./",download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [3]:
X_train, X_test, y_train, y_test = train_test_split(dsmnist.data, dsmnist.targets,test_size=.1,stratify=dsmnist.targets,random_state=400)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=.05, stratify=y_test,random_state=400)

In [4]:
class cls_data(Dataset):
    
    def __init__(self, x, y):
        super().__init__()
        self.data = x
        self.target = y
    def __len__(self):
        return len(self.target)
    def __getitem__(self, index) :
        return self.data[index].unsqueeze(dim=0), self.data[index].flatten() #self.target[index]


In [5]:
ds_train = cls_data(X_train/255., y_train/255.)
ds_test = cls_data(X_test/255., y_test/255.)
ds_val = cls_data(X_val/255.,y_val/255.)

train_loader = DataLoader(ds_train,batch_size=16)
test_loader = DataLoader(ds_test,batch_size=16)
val_loader = DataLoader(ds_val,batch_size=16)

In [6]:
class cls_model(nn.Module):
    def __init__(self) -> None:
        super().__init__()

        self.model_feature = nn.Sequential(OrderedDict([
            ("conv1", nn.Conv2d(in_channels=1,out_channels=3,kernel_size=(3,3))),
            ("maxp1", nn.MaxPool2d(2,2))]))
        
        self.model_classify = nn.Sequential(OrderedDict([
            ("linear1",nn.Linear(13*13*3,1000)),
            ("relu1",nn.ReLU()),
            ("layernorm1",nn.LayerNorm(1000)),
            #("dropout1", nn.Dropout(p=.1)),
            ("linear2",nn.Linear(1000,784))]
        ))
        #self.conv1 = torch.nn.Conv2d(in_channels=1,out_channels=3,kernel_size=(3,3))
        #self.maxp = torch.nn.MaxPool2d(2,2)
        #self.l1 = torch.nn.Linear(13*13*3,500)
        #self.lnorm1 = torch.nn.LayerNorm(500)
        #self.relu1 = torch.nn.ReLU()
        #self.drop1 = torch.nn.Dropout(p=.1)
        #self.l2 = torch.nn.Linear(500,10)

    def forward(self, x):
        #x = torch.permute(x,()
        #x = self.conv1(x)
        #x = self.maxp(x)
        x = self.model_feature.forward(x)

        x = x.reshape((x.shape[0],-1))
        
        y = self.model_classify(x)
        #y = self.l2.forward(self.drop1( self.lnorm1(self.relu1( self.l1.forward(x)))))

        return y

In [7]:
model = cls_model()
model
#model.forward(X_train[:5]/255.)

cls_model(
  (model_feature): Sequential(
    (conv1): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1))
    (maxp1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (model_classify): Sequential(
    (linear1): Linear(in_features=507, out_features=1000, bias=True)
    (relu1): ReLU()
    (layernorm1): LayerNorm((1000,), eps=1e-05, elementwise_affine=True)
    (linear2): Linear(in_features=1000, out_features=784, bias=True)
  )
)

In [8]:
def f_validate(model, dloader, criterion):
    model.eval()
    preds = torch.empty(0)
    targets = torch.empty(0)

    tmpLoss = 0
    for i , (x_batch, y_batch) in enumerate(dloader):
        y_pred = model.forward(x_batch)
        preds = torch.cat((preds, y_pred.argmax(dim=1)))
        targets = torch.cat((targets,y_batch))

        tmpLoss += criterion(y_pred, y_batch).item() / len(y_batch)
        #lst_pred.append( [y_pred.argmax(dim=1),y_batch])
    
    return targets.detach().numpy(), preds.detach().numpy() , tmpLoss

In [9]:
criterion = torch.nn.MSELoss()
optim = torch.optim.SGD(model.parameters(), lr=1e-3)

epochs = 20

dict_res = dict()
for i in range(epochs):
    tmpLoss = 0.
    for j,(x_batch, y_batch) in enumerate(train_loader):

        y_pred = model.forward(x_batch)
        #print(y_pred.shape)
        #print(y_batch.unsqueeze(dim=1).shape)
        loss = criterion(y_pred, y_batch)

        optim.zero_grad()
        loss.backward()
        optim.step()

        tmpLoss += loss.item() / len(y_batch)

        #break
    print("Loss:" , tmpLoss)
    target , pred, valLoss = f_validate(model, val_loader, criterion)
    print("Val loss: ", valLoss)
    dict_res[i] = tmpLoss

Loss: 29.26191450888291
Val loss:  1.7485648393630981
Loss: 15.299369147978723
Val loss:  1.576689558569342
Loss: 14.437266563763842
Val loss:  1.5254402575083077
Loss: 14.087576884077862
Val loss:  1.4969388314057142
Loss: 13.860406411346048
Val loss:  1.4753203662112355
Loss: 13.67262455704622
Val loss:  1.4559536501765251
Loss: 13.496391488472
Val loss:  1.4370126996655017
Loss: 13.319918810622767
Val loss:  1.4176495377905667
Loss: 13.136901170015335
Val loss:  1.3973177357111126
Loss: 12.943661846220493
Val loss:  1.3757557612843812
Loss: 12.738771959673613
Val loss:  1.3529131347313523
Loss: 12.522392124403268
Val loss:  1.328886701958254
Loss: 12.295916037168354
Val loss:  1.3038856564089656
Loss: 12.061700837220997
Val loss:  1.2782092946581542
Loss: 11.82262462680228
Val loss:  1.2521878890693188
Loss: 11.581685757264495
Val loss:  1.2261416984256357
Loss: 11.341691450681537
Val loss:  1.2003469304181635
Loss: 11.105122316395864
Val loss:  1.175048076780513
Loss: 10.8738705611

In [10]:
class cls_data_classification(Dataset):
    
    def __init__(self, x, y):
        super().__init__()
        self.data = x
        self.target = y
    def __len__(self):
        return len(self.target)
    def __getitem__(self, index) :
        return self.data[index].unsqueeze(dim=0), self.target[index]


In [11]:
ds_trainc = cls_data_classification(X_train/255., y_train)
ds_testc = cls_data_classification(X_test/255., y_test)
ds_valc = cls_data_classification(X_val/255.,y_val)

train_loaderc = DataLoader(ds_trainc,batch_size=16)
test_loaderc = DataLoader(ds_testc,batch_size=16)
val_loaderc = DataLoader(ds_valc,batch_size=16)

In [16]:
class cls_model_classification(nn.Module):
    def __init__(self,model) -> None:
        super().__init__()

        
        for param in model.parameters():
            param.requires_grad = False

        self.model_feature = model
        self.model_classify = nn.Sequential(OrderedDict([
            ("linear1",nn.Linear(784,10))]))
        

    def forward(self, x):
        
        x = self.model_feature.forward(x)
        
        y = self.model_classify(x)
        #y = self.l2.forward(self.drop1( self.lnorm1(self.relu1( self.l1.forward(x)))))

        return y

In [17]:
modelc = cls_model_classification(model)

In [18]:
criterion = torch.nn.CrossEntropyLoss()
optim = torch.optim.SGD(modelc.parameters(), lr=1e-3)

epochs = 20

dict_res = dict()
for i in range(epochs):
    tmpLoss = 0.
    for j,(x_batch, y_batch) in enumerate(train_loaderc):

        y_pred = modelc.forward(x_batch)
        #print(y_pred.shape)
        #print(y_batch.unsqueeze(dim=1).shape)
        loss = criterion(y_pred, y_batch)

        optim.zero_grad()
        loss.backward()
        optim.step()

        tmpLoss += loss.item() / len(y_batch)

        #break
    print("Loss:" , tmpLoss)
    target , pred, valLoss = f_validate(modelc, val_loaderc, criterion)
    print("Val acc:", metr.accuracy_score(target,pred), "Val loss: ", valLoss)
    dict_res[i] = tmpLoss

Loss: 51.90633546281606
Val acc: 0.9742105263157895 Val loss:  2.6090118401916698
Loss: 15.995628691947786
Val acc: 0.9754385964912281 Val loss:  2.087326305627357
Loss: 12.277660952982842
Val acc: 0.9763157894736842 Val loss:  1.8930291618453339
Loss: 10.461547950748354
Val acc: 0.9763157894736842 Val loss:  1.7894283287168946
Loss: 9.330427844288351
Val acc: 0.9766666666666667 Val loss:  1.724820542964153
Loss: 8.53834214098606
Val acc: 0.977719298245614 Val loss:  1.680794461266487
Loss: 7.943332586706674
Val acc: 0.9782456140350877 Val loss:  1.6490346718637738
Loss: 7.474838426904171
Val acc: 0.978421052631579 Val loss:  1.6252023347769864
Loss: 7.093277691412368
Val acc: 0.9787719298245614 Val loss:  1.6067996209239936
Loss: 6.774495501023921
Val acc: 0.9789473684210527 Val loss:  1.592282490018988
Loss: 6.50280452734296
Val acc: 0.9787719298245614 Val loss:  1.5806424562179018
Loss: 6.267514908900921
Val acc: 0.9787719298245614 Val loss:  1.5711925176728982
Loss: 6.0610572325749

In [50]:
Image.fromarray( torch.nn.functional.relu( model(ds_test[5][0].unsqueeze(dim=0))*255).type(torch.uint8).numpy().reshape(28,28))

In [51]:
Image.fromarray( torch.nn.functional.relu( ds_test[5][0].unsqueeze(dim=0)*255).type(torch.uint8).numpy().reshape(28,28))

In [19]:
target , pred, testLoss  = f_validate(modelc,test_loaderc,criterion)
metr.accuracy_score(target,pred)

0.9833333333333333

In [ ]:
metr.accuracy_score(targets.detach().numpy(),preds.detach().numpy())*100

97.65

In [ ]:
confusion = metr.confusion_matrix(targets.detach().numpy(),preds.detach().numpy())
confusion

array([[586,   0,   1,   0,   1,   1,   2,   0,   1,   0],
       [  0, 668,   2,   1,   3,   0,   0,   0,   0,   0],
       [  1,   1, 581,   3,   4,   0,   0,   4,   1,   1],
       [  0,   1,   8, 593,   0,   3,   0,   4,   2,   2],
       [  1,   0,   4,   1, 567,   0,   2,   2,   2,   5],
       [  2,   1,   0,   6,   0, 531,   1,   0,   1,   0],
       [  1,   0,   2,   0,   0,   3, 583,   0,   3,   0],
       [  1,   2,   6,   0,   1,   0,   1, 614,   0,   2],
       [  1,   3,   0,   7,   3,   4,   0,   1, 566,   0],
       [  3,   1,   1,   4,   4,   3,   0,   7,   2, 570]])

In [ ]:
Image.fromarray(X_train[22].numpy())

In [ ]:
a=model.model_feature.conv1.forward((X_train[22]/1.0).reshape((1,1,28,28)))